In [ ]:
import dwave.inspector
from dwave.system import DWaveSampler, EmbeddingComposite
from collections import defaultdict
from matplotlib import pyplot as plt
import networkx as nx

# The Maximum Cut Problem

Given a graph $G = (\mathcal{V},\mathcal{E})$, the maximum cut problem lies in partitioning the set of vertices $\mathcal{V}$ into two subsets $\mathcal{S}$ and $\mathcal{T}$ such that the number of edges between the subsets, i.e., the cut size $|\mathcal{C}|$, is maximized. We define the cut size as
$$
    |\mathcal{C}| = \sum_{(i,j)\in\mathcal{E}} \text{cut}_{ij},
$$
where
$$
    \text{cut}_{ij} = 
    \begin{cases}
        0\quad\text{if } (i,j \in\mathcal{S}) \lor (i,j \in\mathcal{T}),
        \\
        1\quad\text{if } (i \in\mathcal{S}\land j\in\mathcal{T}) \lor (i \in\mathcal{T}\land j\in\mathcal{S}).
    \end{cases}
$$
So, the problem reads:
$$
    \max \sum_{(i,j)\in\mathcal{E}} \text{cut}_{ij}
$$

Next, the goal is to represent this problem as an Ising model. To that end, we introduce spin variables $s_i\in\{-1,+1\}$ for each vertex $i$. 

## Step 1
Formulate the problem above as an Ising model.

**Hint:** Remember how the objective function was defined for the "2-Bit Disagree Problem",
$$
    J(s_1,s_2)
    =  
    \frac{1+s_1 s_2}{2}
    =
    \begin{cases}
        0\text{ if }s_1\neq s_2,
        \\
        1\text{ if }s_1=s_2,
    \end{cases}
$$
and connect it to the definition of $\text{cut}_{ij}$.
<details>
    <summary>Solution:</summary>
$$
    \text{cut}_{ij} = \frac{1-s_i s_j}{2}
    =
    \begin{cases}
        0\text{ if }s_1 = s_2 \Leftrightarrow i,j \text{ in the same subset} ,
        \\
        1\text{ if }s_1\neq s_2 \Leftrightarrow i,j \text{ in different subsets}.
    \end{cases}
$$

$$
    \max \sum_{(i,j)\in\mathcal{E}} \text{cut}_{ij} \Leftrightarrow \min - \left(\sum_{(i,j)\in\mathcal{E}} \text{cut}_{ij}\right)
$$

$$
    H_{\text{Ising}} = \sum_{(i,j)\in\mathcal{E}} \frac{s_i s_j}{2}
$$
</details>

## Step 2

The Ising model formulated in spin variables $s_i\in\{-1,+1\}$ can be transformed into a QUBO problem using binary variables $q_i\in\{0,1\}$ by the change of variables:
$$
    s_i = 1 - 2q_i.
$$
Derive how the QUBO matrix $\boldsymbol{Q}$ is constructed.
<details>
    <summary>Solution:</summary>

    
Starting from the contribution from each edge $(i,j)$ in the Ising model and performing the change of variables,
$$
    \frac{s_i s_j}{2} = \frac{1}{2} (1-2q_i)(1-2q_j) = \frac{1}{2} (1-2q_i-2q_j+4q_iq_j) = \frac{1}{2} - q_i -q_j + 2q_i q_j,
$$
the QUBO problem can be defined through the QUBO matrix $\boldsymbol{Q}$:
$$
 H_{\text{QUBO}} = \boldsymbol{q}^T\boldsymbol{Q}\boldsymbol{q},
$$
with contributions from each edge $(i,j)$:
$$
  Q_{ii} = Q_{jj} = -1,\, Q_{ij}= 2.
$$

Note that we ignored the constant energy offset.


</details>

## Note
The following code is based on the example from [https://github.com/dwave-examples/maximum-cut/tree/master](https://github.com/dwave-examples/maximum-cut/tree/master)

## Graph Definition

In [ ]:
# Create an empty graph
G = nx.Graph()

# Add edges to the graph, which automatically adds the nodes
G.add_edges_from([(1,2),(1,3),(2,4),(3,4),(3,5),(4,5)])

## Definition of QUBO Problem

We can provide the QUBO matrix $Q$ as a dictionary. Initially, we create an empty dictionary.

In [ ]:
Q = defaultdict(int)

In the next step, we iterate over all edges in $G$ and add the corresponding contributions to $Q$:

In [ ]:
for i, j in G.edges:
    Q[(i,i)]+= -1
    Q[(j,j)]+= -1
    Q[(i,j)]+= 2

In [ ]:
print(Q)

## Solution of the QUBO Problem

In [ ]:
num_reads = 10
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample_qubo(Q, num_reads=num_reads, label='Maximum Cut Problem')

In [ ]:
print(sampleset)

## Analysis of the Results

In [ ]:
print('-' * 60)
print('{:>15s}{:>15s}{:^15s}{:^15s}'.format('Set 0','Set 1','Energy','Cut Size'))
print('-' * 60)
for sample, E in sampleset.data(fields=['sample','energy']):
    S0 = [k for k,v in sample.items() if v == 0]
    S1 = [k for k,v in sample.items() if v == 1]
    print('{:>15s}{:>15s}{:^15s}{:^15s}'.format(str(S0),str(S1),str(E),str(int(-1*E))))


In [ ]:
lut = sampleset.first.sample

# Interpret the best result in terms of nodes and edges
S0 = [node for node in G.nodes if not lut[node]]
S1 = [node for node in G.nodes if lut[node]]
cut_edges = [(u, v) for u, v in G.edges if lut[u]!=lut[v]]
uncut_edges = [(u, v) for u, v in G.edges if lut[u]==lut[v]]

# Display the best result
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='r', node_size=800)
nx.draw_networkx_nodes(G, pos, nodelist=S1, node_color='c', node_size=800)
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=5)
nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=5)
nx.draw_networkx_labels(G, pos, font_size=20)

filename = "maxcut_plot.png"
plt.savefig(filename, bbox_inches='tight')
print("\nYour plot is saved to {}".format(filename))
plt.close()

In [ ]:
dwave.inspector.show(sampleset)